# Comparative Analysis of Sentiment Analysis Techniques

In this project, we wil draw comparisons between traditional statistical analysis tools and cutting-edge AI/ML models. This notebook will examine of the capabilities of SAS against models such as the BERT transformer and LSTM (Long Short-Term Memory) networks.

## Objective

The primary aim is to evaluate and contrast the effectiveness of each method in classifying sentiments expressed in a dataset composed of various tweets. This evaluation will focus on metrics such as accuracy, recall, and precision scores to evaluate each method's performance.

## Approach

My approach ... (fill in here)

I will start by importing the necessary libraries and connecting to the Twitter comp_dbms PostgreSQL 

In [1]:
# Import packages
import pandas as pd
import psycopg2

In [2]:
# Connect to the database
connection = psycopg2.connect(host='3.230.203.12',
                             user='compdb',
                             port=5438,
                             database='twitter',
                             password='compdbs_postgres')
connection.set_session(readonly=True, autocommit=True)

#From our connection we need a cursor, which acts as our interface into the database
cur = connection.cursor()

In [3]:
res = cur.execute("Select * From twitter.users LIMIT 100")
users = cur.fetchall()
df = pd.DataFrame.from_dict(users)
df.head()

,0,1,2,3,4,5,6,7,8,9,10,11
0,2019-02-01,"4th from the Sun, also known as the Red Planet...",4533,72.0,1091459141397180416,121,The Milkyway Galaxy,Mars,4thFromOurStar,253,None,None
1,2014-12-06,The ocular engine. \nSources: @archillinks\nht...,9599,1.0,2907774137,8781,Meta,Archillect,archillect,319440,https://t.co/l6SZrvMcZV,None
2,2020-05-26,A collection of artifacts of cultural and hist...,702,10128.0,1265155918355980288,805,None,Museum Archive,ArtifactsHub,753,None,None
3,2009-10-28,"Astronaut,Ph.D. Counsel of WFP Japan. Tokyo Ol...",1,16.0,85891683,19093,International Space Station,"NOGUCHI, Soichi 野口 聡－（のぐち そういち）",Astro_Soichi,5645,https://t.co/JgOEk2Thbp,True
4,2009-07-15,The definitive guide to the world's hidden won...,20172,3340.0,57047586,5199,Everywhere,Atlas Obscura,atlasobscura,70779,http://t.co/eRquXCISa8,True
